
# ABCDisCo Double-DisCo Tutorial (Torch + optional PennyLane backend)

This notebook reproduces the **Double-DisCo** training pipeline introduced in [T. Aarrestad *et al.*, *Eur. Phys. J. C* **81**, 1003 (2021), arXiv:2007.14400](https://arxiv.org/abs/2007.14400). It mirrors the reference scripts shipped with this repository and exposes the key stages one-by-one so you can validate the implementation, reproduce the published baselines, and swap the classical discriminant for a Quantum Machine Learning (QML) backend when desired.

> **Mapping to repository scripts**
> - Data ingestion and scaling follow `ABCD_topjets_HLF_DD.py` (lines 69-129) and `data_loader.py` (lines 1-63).
> - Neural-network heads reuse `networks.DNNclassifier` (lines 8-44) while the DisCo penalty mirrors `model_ABCD_2NN.py` (lines 29-120) together with `disco.py` (lines 14-118).
> - Evaluation routines adapt `evaluation.py` (lines 1-141) to produce the ABCD closure and JSD vs. background-rejection metrics discussed in the paper.

The workflow is organised as:

1. **Setup & configuration** (aligns with the hyperparameters used in the Double-DisCo study).
2. **Data loading and preprocessing** (min-max scaling, label/weight bookkeeping).
3. **Model definition** with interchangeable Torch/QML heads.
4. **Training** with the DisCo decorrelation penalty.
5. **Diagnostics & evaluation**: ROC curves, distance correlations, ABCD closure plots, JSD vs. background rejection.
6. **Export** of trained weights and inference scores.

> **Datasets**: The repository already ships reduced CMS top-tagging high-level feature (HLF) samples (`topsample_*_tau.dat.gz`) suitable for this tutorial. You can run the notebook end-to-end without any external downloads.



## Environment preparation

Run the following cell *once per environment* if you still need to install the CPU builds of PyTorch, PennyLane, and analysis utilities. Keep it commented in committed versions to avoid accidental re-installs on shared clusters.


In [ ]:

# Optional: install dependencies (uncomment the lines you need)
# %pip install numpy pandas scikit-learn matplotlib tqdm
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# %pip install pennylane pennylane-lightning
# %pip install pyhf



## 0. Configuration (mirrors `ABCD_topjets_HLF_DD.py` lines 69-129)

We replicate the preprocessing hyperparameters used in the Double-DisCo reference run and expose additional knobs to make rapid CPU tests feasible. Set `FULL_DATASET = True` for a faithful reproduction of the paper-level event counts (requires substantial CPU/GPU time).


In [ ]:

from __future__ import annotations

import gzip
import json
from pathlib import Path
from typing import Dict, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import auc, roc_curve
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

# Repository-local modules (mirror original scripts)
from disco import distance_corr_unbiased
from networks import DNNclassifier

try:
    import pennylane as qml  # optional backend
    PENNYLANE_AVAILABLE = True
except ImportError:  # keep optional
    PENNYLANE_AVAILABLE = False

plt.style.use("seaborn-v0_8-talk")

SEED = 1337
rng = np.random.default_rng(SEED)
torch.manual_seed(SEED)

DATA_ROOT = Path('.')
RAW_FILES = {
    "train": DATA_ROOT / "topsample_train_tau.dat.gz",
    "val": DATA_ROOT / "topsample_val_tau.dat.gz",
    "test": DATA_ROOT / "topsample_test_tau.dat.gz",
}

# Toggle to match the full training statistics used in the paper.
FULL_DATASET = False
EVENT_LIMITS = {
    "train": 50000 if not FULL_DATASET else None,
    "val": 50000 if not FULL_DATASET else None,
    "test": 50000 if not FULL_DATASET else None,
}

BATCH_SIZE = 2048
EPOCHS = 25 if not FULL_DATASET else 200
LEARNING_RATE = 1e-3

# DisCo regularisation weights (cf. model_ABCD_2NN.py lines 29-78)
LAMBDA_MUTUAL = 50.0   # decorrelate the two Double-DisCo scores on background
LAMBDA_MASS = 5.0      # optional: decorrelate each score from jet mass (set to 0.0 to disable)

# ABCD region definitions used for closure checks
S1_CUT = 0.5
S2_CUT = 0.5

# QML backend knobs (used only when BACKEND == "qml")
BACKEND = "torch"  # switch to "qml" after installing PennyLane
N_QUBITS = 6
QML_LAYERS = 2
QML_DEVICE = "default.qubit"

FEATURE_NAMES = [
    "mass",
    "pt",
    "tau1_half",
    "tau2_half",
    "tau3_half",
    "tau1",
    "tau2",
    "tau3",
    "tau4",
    "tau1_sq",
    "tau2_sq",
    "tau3_sq",
    "tau4_sq",
]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")



## 1. Data loading & scaling (`ABCD_topjets_HLF_DD.py` lines 69-87)

The original script concatenates train/validation/test files, performs a global min-max scaling on all high-level features, and appends bookkeeping columns for labels, event weights, bin indices, and masses. We expose the same transformation, add integrity checks, and return a tidy `pandas.DataFrame` for quick inspection.


In [ ]:

def _load_tau_file(path: Path) -> np.ndarray:
    if not path.exists():
        raise FileNotFoundError(f"Missing dataset: {path}")
    with gzip.open(path, 'rt') as handle:
        array = np.loadtxt(handle, delimiter=',', skiprows=15)
    if array.ndim != 2 or array.shape[1] != 14:
        raise ValueError(f"Unexpected shape {array.shape} in {path}")
    return array


def load_and_scale(raw_files: Dict[str, Path]) -> Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray], Dict[str, np.ndarray]]:
    """Replicate the min-max scaling used in `ABCD_topjets_HLF_DD.py`."""
    raw_arrays = {split: _load_tau_file(path) for split, path in raw_files.items()}

    concatenated = np.vstack([arr[:, 1:] for arr in raw_arrays.values()])
    feat_min = concatenated.min(axis=0)
    feat_range = np.maximum(concatenated.max(axis=0) - feat_min, 1e-8)

    features: Dict[str, np.ndarray] = {}
    labels: Dict[str, np.ndarray] = {}
    masses: Dict[str, np.ndarray] = {}

    for split, array in raw_arrays.items():
        scaled = (array[:, 1:] - feat_min) / feat_range
        features[split] = scaled.astype(np.float32)
        labels[split] = array[:, 0].astype(np.float32)
        masses[split] = array[:, 1].astype(np.float32)

    return features, labels, masses


features, labels, masses = load_and_scale(RAW_FILES)

summary = []
for split in ("train", "val", "test"):
    limit = EVENT_LIMITS[split]
    n_events = features[split].shape[0] if limit is None else min(limit, features[split].shape[0])
    summary.append({
        "split": split,
        "events": n_events,
        "signal_fraction": float(labels[split][:n_events].mean()),
    })

summary_df = pd.DataFrame(summary)
display(summary_df)

pd.DataFrame(features["train"][:5], columns=FEATURE_NAMES)



## 2. Torch datasets (`data_loader.py` lines 22-63)

We reproduce the `TopTaggingDataset` logic with a CPU/GPU-agnostic implementation and keep the bookkeeping fields required by the loss function.


In [ ]:

class TorchTopTaggingDataset(Dataset):
    def __init__(self, x: np.ndarray, y: np.ndarray, mass: np.ndarray, weight: np.ndarray | None = None):
        self.x = torch.as_tensor(x, dtype=torch.float32)
        self.y = torch.as_tensor(y, dtype=torch.float32)
        if weight is None:
            weight = np.ones_like(y, dtype=np.float32)
        self.w = torch.as_tensor(weight, dtype=torch.float32)
        self.mass = torch.as_tensor(mass, dtype=torch.float32)

    def __len__(self) -> int:
        return self.x.shape[0]

    def __getitem__(self, idx: int):
        return self.x[idx], self.y[idx], self.w[idx], self.mass[idx]


def _clip_events(split: str, array: np.ndarray) -> np.ndarray:
    limit = EVENT_LIMITS[split]
    if limit is None:
        return array
    return array[:limit]


train_set = TorchTopTaggingDataset(
    _clip_events("train", features["train"]),
    _clip_events("train", labels["train"]),
    _clip_events("train", masses["train"]),
)
val_set = TorchTopTaggingDataset(
    _clip_events("val", features["val"]),
    _clip_events("val", labels["val"]),
    _clip_events("val", masses["val"]),
)
test_set = TorchTopTaggingDataset(
    _clip_events("test", features["test"]),
    _clip_events("test", labels["test"]),
    _clip_events("test", masses["test"]),
)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

len(train_set), len(val_set), len(test_set)



## 3. Model backends (`networks.py` lines 8-78)

We expose two interchangeable heads:

- `TorchDoubleDisco` embeds the exact `DNNclassifier` definition from the reference scripts and is the default.
- `PennyLaneDoubleDisco` provides a minimal variational quantum circuit (VQC) realised with `qml.qnn.TorchLayer`, making it straightforward to prototype QML extensions.


In [ ]:

class TorchDoubleDisco(nn.Module):
    """Wrap two `DNNclassifier` heads exactly as in the Double-DisCo scripts."""

    def __init__(self, n_features: int):
        super().__init__()
        self.head1 = DNNclassifier(n_features, 2)
        self.head2 = DNNclassifier(n_features, 2)

    def forward(self, x: torch.Tensor):
        logits1 = self.head1(x)
        logits2 = self.head2(x)
        score1 = F.softmax(logits1, dim=1)[:, 1]
        score2 = F.softmax(logits2, dim=1)[:, 1]
        return logits1, logits2, score1, score2


class PennyLaneDoubleDisco(nn.Module):
    """Minimal PennyLane VQC head compatible with the Double-DisCo loss."""

    def __init__(self, n_features: int, n_qubits: int = 6, layers: int = 2, device_name: str = "default.qubit"):
        if not PENNYLANE_AVAILABLE:
            raise RuntimeError("PennyLane is not installed. Run `%pip install pennylane pennylane-lightning`.")
        super().__init__()
        self.n_qubits = n_qubits
        self.n_features = n_features

        qdevice = qml.device(device_name, wires=n_qubits)
        weight_shapes = {"weights": (layers, n_qubits)}

        @qml.qnode(qdevice, interface="torch")
        def circuit(inputs, weights):
            x_pad = torch.zeros(n_qubits, dtype=inputs.dtype, device=inputs.device)
            take = min(inputs.shape[-1], n_qubits)
            x_pad[:take] = inputs[..., :take]
            qml.templates.AngleEmbedding(x_pad, wires=range(n_qubits), rotation="Y")
            qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
            return [qml.expval(qml.PauliZ(i)) for i in range(2)]

        self.qlayer = qml.qnn.TorchLayer(circuit, weight_shapes)
        self.head1 = nn.Linear(2, 2)
        self.head2 = nn.Linear(2, 2)

    def forward(self, x: torch.Tensor):
        q_inputs = x[:, : self.n_qubits]
        q_features = self.qlayer(q_inputs)
        logits1 = self.head1(q_features)
        logits2 = self.head2(q_features)
        score1 = F.softmax(logits1, dim=1)[:, 1]
        score2 = F.softmax(logits2, dim=1)[:, 1]
        return logits1, logits2, score1, score2


def build_model(n_features: int) -> nn.Module:
    if BACKEND == "qml":
        model = PennyLaneDoubleDisco(n_features, n_qubits=N_QUBITS, layers=QML_LAYERS, device_name=QML_DEVICE)
    else:
        model = TorchDoubleDisco(n_features)
    return model.to(DEVICE)


model = build_model(len(FEATURE_NAMES))
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
model



## 4. Loss function with DisCo penalty (`model_ABCD_2NN.py` lines 29-78 & `disco.py`)

We compute weighted binary cross-entropy for each head and add:

1. A **mutual decorrelation term** between the two Double-DisCo scores on background events.
2. An optional **mass decorrelation** term per head (toggle via `LAMBDA_MASS`).

The distance-correlation implementation is imported directly from `disco.py`, ensuring parity with the reference code.


In [ ]:

def distance_corr_safe(x: torch.Tensor, y: torch.Tensor, weight: torch.Tensor) -> torch.Tensor:
    if x.numel() <= 2 or y.numel() <= 2:
        return torch.zeros(1, device=x.device, dtype=x.dtype)
    normed = weight / (weight.sum() + 1e-12) * len(weight)
    return distance_corr_unbiased(x, y, normed, power=1)


def compute_losses(model: nn.Module, batch: Tuple[torch.Tensor, ...]) -> Tuple[torch.Tensor, Dict[str, float]]:
    features, labels, weights, masses = batch
    features = features.to(DEVICE)
    labels = labels.to(DEVICE)
    weights = weights.to(DEVICE)
    masses = masses.to(DEVICE)

    logits1, logits2, score1, score2 = model(features)

    loss_cls1 = F.binary_cross_entropy(score1, labels, weight=weights)
    loss_cls2 = F.binary_cross_entropy(score2, labels, weight=weights)
    loss = loss_cls1 + loss_cls2

    metrics = {
        "loss_cls1": float(loss_cls1.detach().cpu()),
        "loss_cls2": float(loss_cls2.detach().cpu()),
    }

    background = labels < 0.5
    if background.any():
        w_bkg = weights[background]
        s1_bkg = score1[background]
        s2_bkg = score2[background]
        m_bkg = masses[background]

        if LAMBDA_MUTUAL > 0.0:
            d_mutual = distance_corr_safe(s1_bkg, s2_bkg, torch.ones_like(w_bkg))
            loss = loss + LAMBDA_MUTUAL * d_mutual
            metrics["dCorr_s1_s2"] = float(d_mutual.detach().cpu())

        if LAMBDA_MASS > 0.0:
            d_mass1 = distance_corr_safe(s1_bkg, m_bkg, torch.ones_like(w_bkg))
            d_mass2 = distance_corr_safe(s2_bkg, m_bkg, torch.ones_like(w_bkg))
            loss = loss + LAMBDA_MASS * (d_mass1 + d_mass2)
            metrics["dCorr_s1_m"] = float(d_mass1.detach().cpu())
            metrics["dCorr_s2_m"] = float(d_mass2.detach().cpu())

    return loss, metrics



## 5. Training loop (`model_ABCD_2NN.py` lines 80-208)

We adapt the original `train`/`val` helpers to work seamlessly on CPU or GPU, collect metrics per epoch, and stop only after the requested number of epochs (no early stopping for clarity).


In [ ]:

def train_one_epoch(model: nn.Module, loader: DataLoader, optimizer: torch.optim.Optimizer) -> Dict[str, float]:
    model.train()
    agg: Dict[str, list[float]] = {}
    for batch in tqdm(loader, leave=False, desc="train"):
        optimizer.zero_grad(set_to_none=True)
        loss, metrics = compute_losses(model, batch)
        loss.backward()
        optimizer.step()
        for key, value in metrics.items():
            agg.setdefault(key, []).append(value)
    return {key: float(np.mean(values)) for key, values in agg.items()}


def evaluate(model: nn.Module, loader: DataLoader) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, Dict[str, float]]:
    model.eval()
    scores1, scores2, labels_all, weights_all, masses_all = [], [], [], [], []
    agg: Dict[str, list[float]] = {}
    with torch.no_grad():
        for batch in tqdm(loader, leave=False, desc="eval"):
            _, metrics = compute_losses(model, batch)
            features, labels, weights, masses = batch
            features = features.to(DEVICE)
            _, _, score1, score2 = model(features)
            scores1.append(score1.cpu().numpy())
            scores2.append(score2.cpu().numpy())
            labels_all.append(labels.numpy())
            weights_all.append(weights.numpy())
            masses_all.append(masses.numpy())
            for key, value in metrics.items():
                agg.setdefault(key, []).append(value)
    scores1 = np.concatenate(scores1)
    scores2 = np.concatenate(scores2)
    labels_all = np.concatenate(labels_all)
    weights_all = np.concatenate(weights_all)
    masses_all = np.concatenate(masses_all)
    metrics_mean = {key: float(np.mean(values)) for key, values in agg.items()}
    return scores1, scores2, labels_all, weights_all, masses_all, metrics_mean


history = []
for epoch in range(EPOCHS):
    train_metrics = train_one_epoch(model, train_loader, optimizer)
    s1_val, s2_val, y_val, w_val, m_val, val_metrics = evaluate(model, val_loader)
    fpr, tpr, _ = roc_curve(y_val, s1_val, sample_weight=w_val)
    roc_auc = auc(fpr, tpr)
    record = {
        "epoch": epoch,
        "roc_auc_s1": roc_auc,
        **{f"train_{k}": v for k, v in train_metrics.items()},
        **{f"val_{k}": v for k, v in val_metrics.items()},
    }
    history.append(record)
    print(f"Epoch {epoch:03d} | AUC(s1)={roc_auc:.3f} | train_loss={train_metrics['loss_cls1']+train_metrics['loss_cls2']:.3f}")

history_df = pd.DataFrame(history)
history_df.tail()



### Training diagnostics

We track the classification losses and distance-correlation penalties to verify convergence.


In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
ax[0].plot(history_df["epoch"], history_df["train_loss_cls1"] + history_df["train_loss_cls2"], label="train")
ax[0].plot(history_df["epoch"], history_df["val_loss_cls1"] + history_df["val_loss_cls2"], label="val")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Binary cross-entropy")
ax[0].legend()
ax[0].set_title("Classification loss")

if "train_dCorr_s1_s2" in history_df:
    ax[1].plot(history_df["epoch"], history_df["train_dCorr_s1_s2"], label="train")
    ax[1].plot(history_df["epoch"], history_df["val_dCorr_s1_s2"], label="val")
    ax[1].set_ylabel("Distance correlation")
    ax[1].set_title("Mutual decorrelation (background)")
    ax[1].legend()
else:
    ax[1].axis('off')

fig.tight_layout()



## 6. Evaluation (`evaluation.py` lines 1-141)

We reproduce the ABCD diagnostics: ROC curves, 2D background score maps, closure ratios, and Jensen-Shannon divergence (JSD) versus background rejection. These metrics match the ones plotted in Fig. 4 of the ABCDisCo paper.


In [ ]:

from evaluation import JSD, JSDvsR


def collect_scores(model: nn.Module, loader: DataLoader):
    model.eval()
    out = []
    with torch.no_grad():
        for features, labels, weights, masses in loader:
            features = features.to(DEVICE)
            _, _, score1, score2 = model(features)
            out.append((score1.cpu().numpy(), score2.cpu().numpy(), labels.numpy(), weights.numpy(), masses.numpy()))
    s1 = np.concatenate([o[0] for o in out])
    s2 = np.concatenate([o[1] for o in out])
    y = np.concatenate([o[2] for o in out])
    w = np.concatenate([o[3] for o in out])
    m = np.concatenate([o[4] for o in out])
    return s1, s2, y, w, m


def abcd_counts(s1, s2, y, w, c1=S1_CUT, c2=S2_CUT):
    mask_b = y < 0.5
    A = (s1 < c1) & (s2 < c2) & mask_b
    B = (s1 >= c1) & (s2 < c2) & mask_b
    C = (s1 < c1) & (s2 >= c2) & mask_b
    D = (s1 >= c1) & (s2 >= c2) & mask_b

    def wsum(mask):
        return float(w[mask].sum())

    return wsum(A), wsum(B), wsum(C), wsum(D)


def scan_abcd(s1, s2, y, w, grid=np.linspace(0.2, 0.95, 20)):
    closure = []
    rejection = []
    for c1 in grid:
        for c2 in grid:
            A, B, C, D = abcd_counts(s1, s2, y, w, c1, c2)
            if A > 0 and B > 0 and C > 0 and D > 0:
                D_pred = (B * C) / A
                closure.append(D_pred / D)
                rejection.append(1.0 / (B / (A + B + C + D)))
    return np.array(rejection), np.array(closure)


s1_te, s2_te, y_te, w_te, m_te = collect_scores(model, test_loader)

fig, ax = plt.subplots(1, 3, figsize=(18, 5))

fpr, tpr, _ = roc_curve(y_te, s1_te, sample_weight=w_te)
ax[0].plot(fpr, tpr, label=f"AUC={auc(fpr, tpr):.3f}")
ax[0].plot([0, 1], [0, 1], linestyle='--', color='grey')
ax[0].set_xlabel("False positive rate")
ax[0].set_ylabel("True positive rate")
ax[0].legend()
ax[0].set_title("ROC (score 1)")

hb = ax[1].hist2d(s1_te[y_te < 0.5], s2_te[y_te < 0.5], bins=40, cmap="viridis")
ax[1].set_xlabel("s1 (background)")
ax[1].set_ylabel("s2 (background)")
ax[1].set_title("Background score map")
fig.colorbar(hb[3], ax=ax[1])

rejection, closure = scan_abcd(s1_te, s2_te, y_te, w_te)
ax[2].scatter(rejection, closure, s=12, alpha=0.6)
ax[2].axhline(1.0, color='k', linestyle='--')
ax[2].set_xscale('log')
ax[2].set_xlabel("Background rejection (1/epsilon_B)")
ax[2].set_ylabel("Closure: D_pred / D_true")
ax[2].set_title("ABCD closure scan")

fig.tight_layout()

sig_mask = y_te > 0.5
bg_mask = y_te < 0.5
metrics = {
    eff: JSDvsR(
        sigscore=s1_te[sig_mask],
        bgscore=s1_te[bg_mask],
        bgmass=s2_te[bg_mask],
        sigweights=w_te[sig_mask],
        bgweights=w_te[bg_mask],
        sigeff=eff,
        minmass=0,
        maxmass=1,
        nbins=40,
    )
    for eff in (10, 30, 50)
}
print("JSD vs R metrics:", json.dumps(metrics, indent=2))

with torch.no_grad():
    s1_t = torch.as_tensor(s1_te, dtype=torch.float32)
    s2_t = torch.as_tensor(s2_te, dtype=torch.float32)
    m_t = torch.as_tensor(m_te, dtype=torch.float32)
    bmask = torch.as_tensor(bg_mask)
    d_s1_m = float(distance_corr_safe(s1_t[bmask], m_t[bmask], torch.ones_like(s1_t[bmask])))
    d_s2_m = float(distance_corr_safe(s2_t[bmask], m_t[bmask], torch.ones_like(s2_t[bmask])))
    d_s1_s2 = float(distance_corr_safe(s1_t[bmask], s2_t[bmask], torch.ones_like(s1_t[bmask])))

print({
    "dCorr(s1, mass | bkg)": d_s1_m,
    "dCorr(s2, mass | bkg)": d_s2_m,
    "dCorr(s1, s2 | bkg)": d_s1_s2,
})



## 7. Persist artefacts

Save inference scores and the trained model weights so downstream scripts (`evaluation.py`, likelihood fits with `pyhf`, etc.) can be run without re-training.


In [ ]:

results_df = pd.DataFrame({
    "s1": s1_te,
    "s2": s2_te,
    "label": y_te,
    "weight": w_te,
    "mass": m_te,
})
results_df.to_parquet("abcdisco_double_disco_scores.parquet", index=False)
torch.save(model.state_dict(), "abcdisco_double_disco_model.pt")
print("Saved abcdisco_double_disco_scores.parquet and abcdisco_double_disco_model.pt")



## 8. Extending to full ABCDisCo & QML studies

- **Recovering the paper-level numbers**: set `FULL_DATASET = True`, increase `EPOCHS` to 200, and tune `LAMBDA_MUTUAL` in the range `[50, 200]` as scanned in `ABCD_topjets_HLF_DD.py` (lines 103-129).
- **QML experiments**: install PennyLane, switch `BACKEND = "qml"`, and adjust `N_QUBITS` / `QML_LAYERS`. The loss and evaluation cells remain unchanged because they operate on the abstract interface shared by both backends.
- **Likelihood fits**: export the `results_df` Parquet file and feed it into the `pyhf`-based closure or limit-setting workflows described in Section 4 of the paper.

> For deeper context, revisit the original ABCDisCo publication (T. Aarrestad *et al.*, *Eur. Phys. J. C* **81**, 1003 (2021), arXiv:2007.14400) and the DisCo decorrelation proposal (M. D. Andrews *et al.*, *Phys. Rev. D* **101**, 094004 (2020), arXiv:1905.08628).
